In [1]:
import pandas as pd
from pathlib import Path
import lxml
from lxml.etree import ElementTree,Element,SubElement
from lxml import etree

In [2]:
def set_xml_values_from_dict(el, a_dict):
    for k, v in a_dict.items():

        subel = el.find(k)
        if subel is not None:
            if v is not None:
                subel.text = str(v)
        else:
            subel = SubElement(el, k)
            subel.text = str(v)

In [3]:
def xml_container_element_to_value_dict(el, convert=True):
    a_dict = dict()
    for sub_el in el.getchildren():
        if len(sub_el.getchildren()) == 0:
            a_dict[sub_el.tag] = sub_el.text
            if convert is True:
                a_dict[sub_el.tag] = convert_xml_text(a_dict[sub_el.tag])
    if 'Index' in a_dict:
        a_dict['Index'] = str(a_dict['Index'])
    return a_dict

def convert_xml_text(text):
    """
    
    Parameters
    ----------
    text : str

    Returns
    -------
    resp : str,int,float
        converted text from xml in native python type.
    """

    if text is None:
        return text
    else:
        if (text.startswith(',') or text.startswith('.')) and text.endswith(tuple([str(el) for el in range(0, 10)])):
            text = '0' + text
        try:

            text_conv = int(text.replace(',', '.'))

        except ValueError:

            try:

                text_conv = float(text.replace(',', '.'))

            except ValueError:

                text_conv = text.replace('\n', '').replace('\t', '')

        return text_conv

In [4]:
path = Path("..")/"modele_donnee"

xml_validator = etree.XMLSchema(file=str((path/"DPE.xsd").absolute()))


df_mdd=pd.read_excel(path/'modele_donnee.xlsx')
df_mdd = df_mdd[['description','nom_variable','parent_xpath','type','condition','DPE NEUF']]

enum_table = pd.read_excel(path/'enum_tables.xlsx', sheet_name=None)

valeur_table = pd.read_excel(path/'valeur_tables.xlsx', sheet_name=None)
valeur_table['coef_masque_lointain_non_homogene']=valeur_table['coef_masque_lointain_non_homoge']
del valeur_table['coef_masque_lointain_non_homoge']
enums=[el for el in df_mdd.nom_variable.dropna().values.tolist() if el.startswith('enum')]
enums_without_id =[el for el in enums if not el.endswith('id')]
enums_without_id


enums=[el for el in df_mdd.nom_variable.dropna().values.tolist() if el.startswith('enum')]
enums_without_id =[el for el in enums if not el.endswith('id')]
enums_without_id

enums_dict = {k:v.set_index('id').lib.to_dict() for k,v in enum_table.items() if 'lib' in v and 'id' in v}



mdd_dict = dict()

for grp in df_mdd.parent_xpath.unique():
    mdd_dict[grp]={el:"" for el in df_mdd.loc[df_mdd.parent_xpath==grp].nom_variable.tolist()}


et=ElementTree(Element('dpe'))
dpe = et.getroot()



In [5]:
with open(r'D:\Cstbdata\MONWORKSPACE\Gitlab\py_3cl_dpe\py_3cl_dpe\json_sortie\cas_test_DHUP\cas_APT_1.json','r') as f:
    appt_1=json.load(f)

# dpe

In [6]:
dpe.set("version","1.0.0")

# administratif

In [7]:
administratif=SubElement(dpe,"administratif")

In [8]:
set_xml_values_from_dict(administratif,{
 'date_visite_diagnostiqueur': '2021-07-10',
 'date_etablissement_dpe':  '2021-07-10',
 'nom_proprietaire': 'commune de Champigny Sur Marne',
 #'nom_proprietaire_installation_commune': "Syndic Saint D'hic",
 'enum_modele_dpe_id': 4,
     "enum_version_id":2,})


In [9]:
diagnostiqueur=SubElement(administratif,"diagnostiqueur")

In [10]:
set_xml_values_from_dict(diagnostiqueur,{
 'usr_logiciel_id': '3',
 'version_logiciel': '3.2',
 'nom_diagnostiqueur': 'Lamar',
 'prenom_diagnostiqueur': 'Kendrick',
 'mail_diagnostiqueur': 'kendrick.lamar@diagnostiqueur.fr',
 'telephone_diagnostiqueur': '0607080910',
 'adresse_diagnostiqueur': '20 avenue de Ségur 75007 Paris',
 'numero_certification_diagnostiqueur': '????????????',
 'organisme_certificateur': 'Organisme Certificateur'})

In [11]:
geolocalisation=SubElement(administratif,"geolocalisation")

In [12]:
set_xml_values_from_dict(geolocalisation,{'invar_logement': '0151234567', 'idpar': '06163000CE8888',
                                          'immatriculation_copropriete': 'AA1234567'})

In [13]:
addr_vars=mdd_dict['dpe.administratif.geolocalisation.adresses.adresse_bien/adresse_proprietaire/adresse_proprietaire_installation_commune']

In [14]:
from geocoding_tools.addok_utils import get_addok_search

## adresses

In [15]:
adresses=SubElement(geolocalisation,"adresses")

### adresse propriétaire

In [16]:
addr =dict()
addr['adresse_brut']='3 rue roland martin'
addr['code_postal_brut']='94500'
addr['nom_commune_brut']="Champigny Sur Marne"
addr['label_brut']=' '.join([addr['adresse_brut'],addr['code_postal_brut'],addr['nom_commune_brut']])
addr['enum_statut_geocodage_ban_id']=1
addr['ban_date_appel']="2021-07-10"

# addr.update({'compl_nom_residence': 'résidence les lilas',
#  'compl_ref_batiment': 'A',
#  'compl_etage_appartement': 0,
#  'compl_ref_cage_escalier': 'A1',
#  'compl_ref_logement': 'A12'})

addr_res_dict=get_addok_search(addr['label_brut'])[0]['properties']
addr_res_dict={'ban_'+k:v for k,v in addr_res_dict.items()}

for k,v in addr_vars.items():
    if k in addr_res_dict:
        addr[k]=addr_res_dict[k]

In [17]:
adresse_proprietaire=SubElement(adresses,"adresse_proprietaire")

In [18]:
set_xml_values_from_dict(adresse_proprietaire,addr)

In [19]:
addr

{'adresse_brut': '3 rue roland martin',
 'code_postal_brut': '94500',
 'nom_commune_brut': 'Champigny Sur Marne',
 'label_brut': '3 rue roland martin 94500 Champigny Sur Marne',
 'enum_statut_geocodage_ban_id': 1,
 'ban_date_appel': '2021-07-10',
 'ban_id': '94017_8243_00003',
 'ban_label': '3 Rue Roland Martin 94500 Champigny-sur-Marne',
 'ban_housenumber': '3',
 'ban_street': 'Rue Roland Martin',
 'ban_citycode': '94017',
 'ban_postcode': '94500',
 'ban_city': 'Champigny-sur-Marne',
 'ban_type': 'housenumber',
 'ban_score': 0.9665599999999999,
 'ban_x': 662242.87,
 'ban_y': 6857973.84}

In [20]:
addr = dict()

addr['adresse_brut']='116 avenue de Rosny'
addr['code_postal_brut']='93130'
addr['nom_commune_brut']="Noisy Le Sec"
addr['label_brut']=' '.join([addr['adresse_brut'],addr['code_postal_brut'],addr['nom_commune_brut']])
addr['enum_statut_geocodage_ban_id']=1
addr['ban_date_appel']="2021-07-10"

# addr.update({'compl_nom_residence': 'résidence des iris',
#  'compl_ref_batiment': 'B',
#  'compl_etage_appartement':3,
#  'compl_ref_logement': '40'})

addr_res_dict=get_addok_search(addr['label_brut'])[0]['properties']
addr_res_dict={'ban_'+k:v for k,v in addr_res_dict.items()}


for k,v in addr_vars.items():
    if k in addr_res_dict:
        addr[k]=addr_res_dict[k]

In [21]:
adresse_bien=SubElement(adresses,"adresse_bien")

In [22]:
set_xml_values_from_dict(adresse_bien,addr)

# modele tertiaire

In [23]:
tertiaire=SubElement(dpe,"tertiaire")
caracteristique_generale=SubElement(tertiaire,"caracteristique_generale")
set_xml_values_from_dict(caracteristique_generale,
{'enum_usage_fonctionnel_batiment_id': 25,
 'enum_categorie_erp_id': 1,
 'surface_utile': 2500,
 'shon': 2500*1.3,
 'nombre_occupant': 100,
 'enum_periode_construction_id': 3,
 'annee_construction': 1976})

bilan_consommation=SubElement(tertiaire,"bilan_consommation")

set_xml_values_from_dict(bilan_consommation,
{'classe_conso_energie': 'D',
 'classe_emission_ges': 'D',
 'conso_energie_primaire': 210,
 'emission_ges': 30})


consommation_collection=SubElement(tertiaire,"consommation_collection")
consommation=SubElement(consommation_collection,"consommation")

set_xml_values_from_dict(consommation,{'conso_energie_finale': 150*2500,
 'conso_energie_primaire': 250*2500,
 'enum_type_usage_id':11 ,
 'enum_type_energie_id': 2,
 'frais_annuels_energie': 150*2500*0.05,
'annee_consommation':2017
                                      })

set_xml_values_from_dict(consommation,{'conso_energie_finale': 70*2500,
 'conso_energie_primaire': 2.58*70*2500,
 'enum_type_usage_id':7 ,
 'enum_type_energie_id': 1,
 'frais_annuels_energie': 70*2500*0.11,
 'annee_consommation':2017
                                     
                                      })

# descriptifs et fiches techniques

In [24]:
descriptif_enr_collection=SubElement(dpe,"descriptif_enr_collection")


In [25]:
descriptif_simplifie_collection=SubElement(dpe,"descriptif_simplifie_collection")


In [26]:
descriptif_simplifie=SubElement(descriptif_simplifie_collection,"descriptif_simplifie")
set_xml_values_from_dict(descriptif_simplifie,
{'description': "mur sud en briques pleines doubles avec lame d'air non isolé; mur est en briques pleines simples donnant sur une circulation",
 'enum_categorie_descriptif_simplifie_id':1})

descriptif_simplifie=SubElement(descriptif_simplifie_collection,"descriptif_simplifie")
set_xml_values_from_dict(descriptif_simplifie,{'description': 'plancher intermédiaire non déperditif donnant sur un autre logement adjacent',
                                                        'enum_categorie_descriptif_simplifie_id': 2})

descriptif_simplifie=SubElement(descriptif_simplifie_collection,"descriptif_simplifie")
set_xml_values_from_dict(descriptif_simplifie,{'description': 'plancher intermédiaire non déperditif donnant sur un autre logement adjacent',
                                                        'enum_categorie_descriptif_simplifie_id': 3})

descriptif_simplifie=SubElement(descriptif_simplifie_collection,"descriptif_simplifie")
set_xml_values_from_dict(descriptif_simplifie,
                         {'description': 'porte en bois pleine simple; Fenêtre battante bois simple vitrage orienté sud;Porte Fenêtre bois simple vitrage orienté sud',
                          'enum_categorie_descriptif_simplifie_id': 4})

descriptif_simplifie=SubElement(descriptif_simplifie_collection,"descriptif_simplifie")
set_xml_values_from_dict(descriptif_simplifie,{'description': 'convecteurs electriques anciens(système individuel)',
                                                        'enum_categorie_descriptif_simplifie_id': 5})

descriptif_simplifie=SubElement(descriptif_simplifie_collection,"descriptif_simplifie")
set_xml_values_from_dict(descriptif_simplifie,{'description': "ballon éléctrique vertical 300L", 
                                                        'enum_categorie_descriptif_simplifie_id': 6})

descriptif_simplifie=SubElement(descriptif_simplifie_collection,"descriptif_simplifie")
set_xml_values_from_dict(descriptif_simplifie,{'description': 'Sans Object', 'enum_categorie_descriptif_simplifie_id': 7})

descriptif_simplifie=SubElement(descriptif_simplifie_collection,"descriptif_simplifie")
set_xml_values_from_dict(descriptif_simplifie,{'description': 'ventilation naturelle par conduit', 'enum_categorie_descriptif_simplifie_id': 8})


In [27]:
fiche_technique_collection=SubElement(dpe,"fiche_technique_collection")

fiche_technique=SubElement(fiche_technique_collection,"fiche_technique")
set_xml_values_from_dict(fiche_technique,{'enum_categorie_fiche_technique_id': 1})


sous_fiche_technique_collection=SubElement(fiche_technique,"sous_fiche_technique_collection")


for description,enum_origine_donnee_id,valeur in [
  ('surface',2,'80m²'),
  ("type d'adjacence",2,'extérieur'),
('matériau mur',2,'briques pleines doubles'),
    ('état isolation',3,'non isolé'),
]:
    sous_fiche_technique=SubElement(sous_fiche_technique_collection,"sous_fiche_technique")
    set_xml_values_from_dict(sous_fiche_technique,
    {'description': description,
     'enum_origine_donnee_id': enum_origine_donnee_id,
    'valeur':valeur
    })


fiche_technique=SubElement(fiche_technique_collection,"fiche_technique")
set_xml_values_from_dict(fiche_technique,{'enum_categorie_fiche_technique_id': 1})
sous_fiche_technique_collection=SubElement(fiche_technique,"sous_fiche_technique_collection")

for description,enum_origine_donnee_id,valeur in [
  ('surface',2,'51m²'),
  ("type d'adjacence",2,'extérieur'),
('matériau mur',2,'bloc de béton plein'),
    ('état isolation',3,'isolé (ITI)'),
]:

    sous_fiche_technique=SubElement(sous_fiche_technique_collection,"sous_fiche_technique")
    set_xml_values_from_dict(sous_fiche_technique,
    {'description': description,
     'enum_origine_donnee_id': enum_origine_donnee_id,
    'valeur':valeur
    })



# fiche_technique=SubElement(fiche_technique_collection,"fiche_technique")
# set_xml_values_from_dict(fiche_technique,{'enum_categorie_fiche_technique_id': 1})


# sous_fiche_technique_collection=SubElement(fiche_technique,"sous_fiche_technique_collection")
# sous_fiche_technique=SubElement(sous_fiche_technique_collection,"sous_fiche_technique")
# set_xml_values_from_dict(sous_fiche_technique,{'description': "mur est en briques pleines simple non isolé donnant sur un immeuble a usage autre qu'habitation U = 3.5 W/m².K, b=0.2 S = 15m²",
#  'enum_origine_donnee_id': 2})



fiche_technique=SubElement(fiche_technique_collection,"fiche_technique")
set_xml_values_from_dict(fiche_technique,{'enum_categorie_fiche_technique_id': 4})

sous_fiche_technique_collection=SubElement(fiche_technique,"sous_fiche_technique_collection")
"Fenêtre battante orientée sud simple vitrage Ug = 5.8 W/m².K,Uw = 4.7 W/m².K, b=1"
for description,enum_origine_donnee_id,valeur in [
  ('surface',2,'5m²'),
  ("type d'adjacence",2,'extérieur'),
('type vitrage',2,'Double vitrage'),
]:
    sous_fiche_technique=SubElement(sous_fiche_technique_collection,"sous_fiche_technique")
    set_xml_values_from_dict(sous_fiche_technique,
    {'description': description,
     'enum_origine_donnee_id': enum_origine_donnee_id,
    'valeur':valeur
    })


# fiche_technique=SubElement(fiche_technique_collection,"fiche_technique")
# set_xml_values_from_dict(fiche_technique,{'enum_categorie_fiche_technique_id': 4})


# sous_fiche_technique_collection=SubElement(fiche_technique,"sous_fiche_technique_collection")
# sous_fiche_technique=SubElement(sous_fiche_technique_collection,"sous_fiche_technique")
# set_xml_values_from_dict(sous_fiche_technique,{'description': "Porte Fenêtre battante orientée sud simple vitrage équipé d'une jalousie accordéon Ug = 5.8 W/m².K,Uw = 4.7,Ujn = 3.8 W/m².K, b=1",
#            "detail_origine_donnee":"mesure du type de vitrage au vitromètre",                                    
#  'enum_origine_donnee_id': 2})


# fiche_technique=SubElement(fiche_technique_collection,"fiche_technique")
# set_xml_values_from_dict(fiche_technique,{'enum_categorie_fiche_technique_id': 4})


# sous_fiche_technique_collection=SubElement(fiche_technique,"sous_fiche_technique_collection")
# sous_fiche_technique=SubElement(sous_fiche_technique_collection,"sous_fiche_technique")
# set_xml_values_from_dict(sous_fiche_technique,{'description': "Porte Fenêtre battante orientée sud simple vitrage équipé d'une jalousie accordéon Ug = 5.8 W/m².K,Uw = 4.7,Ujn = 3.8 W/m².K, b=1",
#            "detail_origine_donnee":"mesure du type de vitrage au vitromètre",                                    
#  'enum_origine_donnee_id': 2})


# fiche_technique=SubElement(fiche_technique_collection,"fiche_technique")
# set_xml_values_from_dict(fiche_technique,{'enum_categorie_fiche_technique_id': 5})


# sous_fiche_technique_collection=SubElement(fiche_technique,"sous_fiche_technique_collection")
# sous_fiche_technique=SubElement(sous_fiche_technique_collection,"sous_fiche_technique")
# set_xml_values_from_dict(sous_fiche_technique,{'description': "Porte Simple en bois U = 3.5 W/m²/K",
# 'enum_origine_donnee_id': 2})



# fiche_technique=SubElement(fiche_technique_collection,"fiche_technique")
# set_xml_values_from_dict(fiche_technique,{'enum_categorie_fiche_technique_id': 6})


# sous_fiche_technique_collection=SubElement(fiche_technique,"sous_fiche_technique_collection")
# sous_fiche_technique=SubElement(sous_fiche_technique_collection,"sous_fiche_technique")
# set_xml_values_from_dict(sous_fiche_technique,{'description': "Pont thermique refend/mur l= 2.8 k=0.365 W/m/K",
# 'enum_origine_donnee_id': 2})


# fiche_technique=SubElement(fiche_technique_collection,"fiche_technique")
# set_xml_values_from_dict(fiche_technique,{'enum_categorie_fiche_technique_id': 6})


# sous_fiche_technique_collection=SubElement(fiche_technique,"sous_fiche_technique_collection")
# sous_fiche_technique=SubElement(sous_fiche_technique_collection,"sous_fiche_technique")
# set_xml_values_from_dict(sous_fiche_technique,{'description': "Pont thermique refend/mur l= 2.8 k=0.365 W/m/K",
# 'enum_origine_donnee_id': 2})

# fiche_technique=SubElement(fiche_technique_collection,"fiche_technique")
# set_xml_values_from_dict(fiche_technique,{'enum_categorie_fiche_technique_id': 6})


# sous_fiche_technique_collection=SubElement(fiche_technique,"sous_fiche_technique_collection")
# sous_fiche_technique=SubElement(sous_fiche_technique_collection,"sous_fiche_technique")
# set_xml_values_from_dict(sous_fiche_technique,{'description': "Pont thermique fenetre/mur l= 5 k=0.25 W/m/K",
# 'enum_origine_donnee_id': 2})

# fiche_technique=SubElement(fiche_technique_collection,"fiche_technique")
# set_xml_values_from_dict(fiche_technique,{'enum_categorie_fiche_technique_id': 6})

# sous_fiche_technique_collection=SubElement(fiche_technique,"sous_fiche_technique_collection")
# sous_fiche_technique=SubElement(sous_fiche_technique_collection,"sous_fiche_technique")
# set_xml_values_from_dict(sous_fiche_technique,{'description': "Pont thermique pf/mur l= 21 k=0.365 W/m/K",
# 'enum_origine_donnee_id': 2})

# fiche_technique=SubElement(fiche_technique_collection,"fiche_technique")
# set_xml_values_from_dict(fiche_technique,{'enum_categorie_fiche_technique_id': 6})

# sous_fiche_technique_collection=SubElement(fiche_technique,"sous_fiche_technique_collection")
# sous_fiche_technique=SubElement(sous_fiche_technique_collection,"sous_fiche_technique")
# set_xml_values_from_dict(sous_fiche_technique,{'description': 'pont thermique Plancher intermediaire/mur l= 16 k=0.43 W/m/K',
# 'enum_origine_donnee_id': 2})



# fiche_technique=SubElement(fiche_technique_collection,"fiche_technique")
# set_xml_values_from_dict(fiche_technique,{'enum_categorie_fiche_technique_id': 7})

# sous_fiche_technique_collection=SubElement(fiche_technique,"sous_fiche_technique_collection")
# sous_fiche_technique=SubElement(sous_fiche_technique_collection,"sous_fiche_technique")
# set_xml_values_from_dict(sous_fiche_technique,{'description': 'convecteurs electriques anciens Rg=1 Re=0.95 Rr=0.9 Rd=1',
# 'enum_origine_donnee_id': 2})

# fiche_technique=SubElement(fiche_technique_collection,"fiche_technique")
# set_xml_values_from_dict(fiche_technique,{'enum_categorie_fiche_technique_id': 8})

# sous_fiche_technique_collection=SubElement(fiche_technique,"sous_fiche_technique_collection")
# sous_fiche_technique=SubElement(sous_fiche_technique_collection,"sous_fiche_technique")
# set_xml_values_from_dict(sous_fiche_technique,{'description': 'ballon electrique vertical atlantic zeneo 300L Rg=1 Rs=0.64 Rd=0.93',
# 'enum_origine_donnee_id': 3})

# fiche_technique=SubElement(fiche_technique_collection,"fiche_technique")
# set_xml_values_from_dict(fiche_technique,{'enum_categorie_fiche_technique_id': 10})

# sous_fiche_technique_collection=SubElement(fiche_technique,"sous_fiche_technique_collection")
# sous_fiche_technique=SubElement(sous_fiche_technique_collection,"sous_fiche_technique")
# set_xml_values_from_dict(sous_fiche_technique,{'description': 'Ventilation naturelle par conduit ',
# 'enum_origine_donnee_id': 2})

# justificatifs

In [28]:
justificatif_collection=SubElement(dpe,"justificatif_collection")
justificatif=SubElement(justificatif_collection,"justificatif")
set_xml_values_from_dict(justificatif,
{'description': "notice technique du ballon d'eau chaude atlantic zeneo", 'enum_type_justificatif_id':10 })

# gestes entretien

In [29]:
descriptif_geste_entretien_collection=SubElement(dpe,"descriptif_geste_entretien_collection")
descriptif_geste_entretien=SubElement(descriptif_geste_entretien_collection,"descriptif_geste_entretien")
set_xml_values_from_dict(descriptif_geste_entretien,
{'enum_picto_geste_entretien_id': 9,
 'description': "detarter et vidanger le ballon d'eau chaude tous les 5 ans",
 'categorie_geste_entretien': 'entretien du chauffe-eau'})

# pack travaux

In [30]:
descriptif_travaux = SubElement(dpe,'descriptif_travaux')
commentaire_travaux = SubElement(descriptif_travaux,'commentaire_travaux')
commentaire_travaux.text = """Des travaux peuvent vous permettre d’améliorer significativement l’efficacité énergétique de votre
logement et ainsi de faire des économies d’énergie, d’améliorer son confort, de le valoriser et de le
rendre plus écologique. Le pack de travaux vous permet de réaliser les travaux prioritaires,
et le pack d’aller vers un logement très performant."""

pack_travaux_collection=SubElement(descriptif_travaux,"pack_travaux_collection")
pack_travaux=SubElement(pack_travaux_collection,"pack_travaux")
set_xml_values_from_dict(pack_travaux,
{'enum_num_pack_travaux_id': 1,
 'conso_5_usages_apres_travaux': 99,
 'cout_pack_travaux_min': 999,
 'cout_pack_travaux_max': 9999})

travaux_collection=SubElement(pack_travaux,"travaux_collection")
travaux = SubElement(travaux_collection,'travaux')
set_xml_values_from_dict(travaux,
{'enum_lot_travaux_id': 1,
 'description_travaux': 'isolation murs ITE 15 cm',
 'avertissement_travaux': 'permis de construire',
 'performance_recommande': 'R = 3,75 m2. K/W'})

travaux = SubElement(travaux_collection,'travaux')
set_xml_values_from_dict(travaux,
{'enum_lot_travaux_id': 4,
 'description_travaux': 'remplacement des vitrages par du double vitrage performant',
 'performance_recommande': 'U = 2.4 W/m²/K'})
                         
pack_travaux=SubElement(pack_travaux_collection,"pack_travaux")
set_xml_values_from_dict(pack_travaux,
{'enum_num_pack_travaux_id': 2,
 'conso_5_usages_apres_travaux': 99,
 'cout_pack_travaux_min': 999,
 'cout_pack_travaux_max': 9999})

travaux_collection=SubElement(pack_travaux,"travaux_collection")
travaux = SubElement(travaux_collection,'travaux')
set_xml_values_from_dict(travaux,
{'enum_lot_travaux_id':5,
 'description_travaux': 'remplacement du système de chauffage par une pompe à chaleur',
 'avertissement_travaux': 'permis de construire',
 'performance_recommande': 'SCOP=4.2'})

                         
                         
pack_travaux=SubElement(pack_travaux_collection,"pack_travaux")
set_xml_values_from_dict(pack_travaux,
{'enum_num_pack_travaux_id': 3,
 'conso_5_usages_apres_travaux': 99,
 'cout_pack_travaux_min': 999,
 'cout_pack_travaux_max': 9999})

travaux_collection=SubElement(pack_travaux,"travaux_collection")
travaux = SubElement(travaux_collection,'travaux')
set_xml_values_from_dict(travaux,
{'enum_lot_travaux_id': 1,
 'description_travaux': 'isolation murs ITE 15 cm',
 'avertissement_travaux': 'permis de construire',
 'performance_recommande': 'R = 3,75 m2. K/W'})
                         
travaux = SubElement(travaux_collection,'travaux')
set_xml_values_from_dict(travaux,
{'enum_lot_travaux_id': 4,
 'description_travaux': 'remplacement des vitrages par du double vitrage performant',
 'performance_recommande': 'U = 2.4 W/m²/K'})
                         
                         
travaux = SubElement(travaux_collection,'travaux')
set_xml_values_from_dict(travaux,
{'enum_lot_travaux_id':5,
 'description_travaux': 'remplacement du système de chauffage par une pompe à chaleur',
 'avertissement_travaux': 'permis de construire',
 'performance_recommande': 'SCOP=4.2'})


# valid

In [31]:
et.write(str((path/'exemples_metier'/'cas_test_tertiaire_1_valid.xml')), pretty_print=True, xml_declaration=True, encoding='utf-8')

In [32]:
import lxml

In [33]:


is_valid = xml_validator.validate(et)

In [34]:
xml_validator.error_log

In [35]:
is_valid

True

In [36]:
import requests

dpe.find('*enum_version_id').text='2'

r=requests.post('http://localhost:5000/controle_coherence',data=etree.tostring(dpe))

In [37]:
r.json()

{'erreur_logiciel': [],
 'erreur_saisie': [],
 'message_principal': 'CONTROLE DE COHERENCE DU MODELE NON EFFECTUE SUR DPE TERTIAIRE.',
 'notification': [],
 'validation_xsd': {'error_log': [''], 'valid': True},
 'warning_logiciel': [],
 'warning_saisie': []}

In [38]:
logging.

SyntaxError: invalid syntax (<ipython-input-38-7eff0fb4b152>, line 1)